##### AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<center>
    <h1 style = "font-size:48px; font-weight:normal"><b>AI</b>-TECH</style></h1>
    <br>
<br>
<br>
<br>
    <h1 style = "font-size:24px; font-weight:normal">LABORATORIUM 4</style></h1>
    <h1 style = "font-size:24px">z Głębokiego przetwarzanie tekstu i mowy</style></h1>
    <h1 style = "font-size:24px">NLP z wykorzystaniem sieci GPT
2 lub GPT 3 , cz . 1</style></h1>
    <h1 style = "font-size:24px">Przemysław Rośleń</style></h1>
    <h3 style =
</center>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
    </body>
    </html>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

<hr>

# Trening sieci GPT-2 w TensorFlow 2.0


Niniejszy zawiera procedury do treningu sieci GPT-2 w bibliotece TensorFlow 2.0

In [ ]:
# Instalacja biblioteki transformers - domyślnie niestety maszyny Colaba nie posiadają jej zainstalowanej:

!pip install transformers

In [ ]:
# Import bibliotek
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import pipeline, set_seed
import numpy as np
import os
import tensorflow as tf

# Przydatne źródła:
# https://data-dive.com/finetune-german-gpt2-on-tpu-transformers-tensorflow-for-text-generation-of-reviews
# https://github.com/huggingface/transformers/issues/2439

# Definicja modeli:

W razie, gdy modelu nie ma na maszynie - zostanie on pobrany z serwerów firmy utrzymującej bibliotekę transformers.

In [ ]:
# Do wyboru są różne rozmiary modelu GPT-2:
# "gpt2"
# "gpt2-medium"
# "gpt2-large"
# "gpt2-xl"
# "distilgpt2" - wersja kompaktowa uzyskana w wyniku procesu tzw. destylacji wiedzy

model     = TFGPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Pobranie danych treningowych

In [ ]:
# Odczyt danych treningowych z pliku. Należy utworzyć go przed treningiem dobierając
# jako jego treść teksty o pożądanym charakterze.
file_path = 'text.txt'
with open(file_path, encoding="utf-8") as f:
    text = f.read()

# Tokenizacja odczytanych danych:
tokenized_text = tokenizer.encode(text)

# Przygotowanie zbioru treningowego (setting modelowania języka - ang. language modeling)
examples   = []
block_size = 100
for i in range(0, len(tokenized_text) - block_size + 1, block_size):
    examples.append(tokenized_text[i:i + block_size])

inputs = []
labels = []
for example in examples:
    inputs.append(example[:-1])
    labels.append(example[1:])

dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))

# Tutaj zaczyna się przygotowanie samego procesu treningu:
BATCH_SIZE  = 12
BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Trening sieci

In [ ]:
# Przygotowanie pomocniczych obiektów i kompilacja modelu.
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric    = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

print()
print("Przykład tekstu wygenerowanego przez sieć przed treningiem:")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
generated_output = generator("", max_length=100, num_return_sequences=1)
print(generated_output[0]['generated_text'])
print()

# Trening w nieskończonej pętli - z podglądem wyjścia sieci po każdej epoce
# treningu.
epoch_number = 1
while True:
    print(f"numer epoki: {epoch_number}")
    model.fit(dataset, epochs=1)
    generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
    generated_output = generator("", max_length=100, num_return_sequences=1)
    print(generated_output[0]['generated_text'])
    print()
    epoch_number += 1
